<img src='https://raw.githubusercontent.com/autonomio/hyperio/master/logo.png' width=250px>

This notebook is a compliment to the *Hyperparameter Optimization on Keras* article. 

## Overview

There are four steps to setting up an experiment with Talos:

1) Imports and data

2) Creating the Keras model

3) Defining the Parameter Space Boundaries 

4) Running the Experiment

## 1. The Required Inputs and Data

In [2]:
!pip install talos

     |████████████████████████████████| 61kB 3.5MB/s 
     |████████████████████████████████| 5.4MB 6.8MB/s 
     |████████████████████████████████| 9.5MB 52.5MB/s 
     |████████████████████████████████| 2.6MB 27.5MB/s 
  Created wheel for kerasplotlib: filename=kerasplotlib-0.1.6-cp37-none-any.whl size=3603 sha256=b16f337e7adf86cd8a0ae5f6e010ec6e058c6055742d52de9251f373caa02195
  Stored in directory: /root/.cache/pip/wheels/9d/d3/8c/9503a22b0a38e8b21c70ad834e4606d209193443e5c709305d
  Created wheel for wrangle: filename=wrangle-0.6.7-cp37-none-any.whl size=49894 sha256=62d271dff705b59f5eb97b07cc0fa2132af87c7787f4dbacb3f209b361f4ba99
  Stored in directory: /root/.cache/pip/wheels/bf/1b/50/d0403ce6ef269e364894da7b50db68db14c4ac62c577561e2d
  Created wheel for chances: filename=chances-0.1.9-cp37-none-any.whl size=41610 sha256=6b55c50a58dc9de2d3fccda880edc0fabe553310c982cf006f92039aa914d269
  Stored in directory: /root/.cache/pip/wheels/75/33/46/c871b94249bd57d17797d049b3dff8e3a09c315af

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense

%matplotlib inline

import sys
sys.path.insert(0, '/Users/mikko/Documents/GitHub/talos')
import talos

In [4]:
# then we load the dataset
x, y = talos.templates.datasets.breast_cancer()

# and normalize every feature to mean 0, std 1
x = talos.utils.rescale_meanzero(x)

## 2. Creating the Keras Model

In [5]:
# first we have to make sure to input data and params into the function
def breast_cancer_model(x_train, y_train, x_val, y_val, params):

    model = Sequential()
    model.add(Dense(params['first_neuron'], input_dim=x_train.shape[1],
                    activation=params['activation'],
                    kernel_initializer=params['kernel_initializer']))
    
    model.add(Dropout(params['dropout']))

    model.add(Dense(1, activation=params['last_activation'],
                    kernel_initializer=params['kernel_initializer']))
    
    model.compile(loss=params['losses'],
                  optimizer=params['optimizer'],
                  metrics=['acc', talos.utils.metrics.f1score])
    
    history = model.fit(x_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        callbacks=[talos.utils.live()],
                        epochs=params['epochs'],
                        verbose=0)

    return history, model

## 3. Defining the Parameter Space Boundary

In [6]:
# then we can go ahead and set the parameter space
p = {'first_neuron':[9,10,11],
     'hidden_layers':[0, 1, 2],
     'batch_size': [30],
     'epochs': [100],
     'dropout': [0],
     'kernel_initializer': ['uniform','normal'],
     'optimizer': ['Nadam', 'Adam'],
     'losses': ['binary_crossentropy'],
     'activation':['relu', 'elu'],
     'last_activation': ['sigmoid']}

## 4. Starting the Experiment

In [7]:
# and run the experiment
t = talos.Scan(x=x,
               y=y,
               model=breast_cancer_model,
               params=p,
               experiment_name='breast_cancer',
               round_limit=10)

  0%|          | 0/10 [00:00<?, ?it/s]

KeyError: ignored